In [1]:
import pandas as pd
import numpy as np
from astropy.cosmology import Planck18 as cosmo
from astropy import units as u
import requests
from io import StringIO

In [19]:
import pandas as pd
import requests
from io import StringIO

# Step 1: Download the file
url = "https://content.cld.iop.org/journals/0004-637X/693/2/1484/revision1/apj299693t1_mrt.txt"
response = requests.get(url)
lines = response.text.splitlines()

# Step 2: Extract only data lines
data_lines = [line for line in lines if line.startswith("GRB")]
data_str = "\n".join(data_lines)

# Step 3: Define column specs and names (just the ones you want)
colspecs = [
    (0, 10),   # GRB name
    (30, 40),  # Time since burst [s]
    (41, 45),  # Filter
    (53, 54),  # Limit flag
    (54, 59),  # Magnitude
    (60, 64),  # Magnitude error
    (65, 66),  # Mag reference
]

colnames = ["GRB", "Time_sec", "Filter", "LimitFlag", "App_Mag", "Mag_Err", "Mag_Ref"]

# Step 4: Read into DataFrame
df = pd.read_fwf(StringIO(data_str), colspecs=colspecs, names=colnames)

# Step 5: Show preview
print(df.head(20))


           GRB  Time_sec Filter LimitFlag  App_Mag  Mag_Err  Mag_Ref
0    GRB050412     391.0   R_C_         >    20.80      NaN        1
1    GRB050412     684.7   R_C_         >    21.40      NaN        1
2    GRB050412    1063.7      i         >    21.00      NaN        1
3    GRB050412    1646.0      z         >    20.20      NaN        1
4    GRB050412    2858.8   R_C_         >    22.50      NaN        1
5    GRB050412    2817.0      i         >    22.00      NaN        1
6   GRB050416A     216.0   I_C_       NaN    18.82     0.11        2
7   GRB050416A     371.5   I_C_       NaN    18.86     0.11        2
8   GRB050416A     527.0   I_C_       NaN    19.16     0.13        2
9   GRB050416A     691.2   I_C_       NaN    19.35     0.23        2
10  GRB050416A     846.7   I_C_       NaN    19.01     0.12        2
11  GRB050416A    1002.2   I_C_       NaN    19.53     0.18        2
12  GRB050416A    1347.8      z       NaN    19.16     0.27        2
13  GRB050416A    1874.9   I_C_   

In [29]:
import pandas as pd
import numpy as np
import re

# --- Load and clean Table 1 photometry ---
with open("apj299693t1_mrt.txt", "r") as f:
    lines1 = f.readlines()

# Extract only the data lines
data_lines1 = [line for line in lines1 if line.startswith("GRB")]
table1 = []

for line in data_lines1:
    parts = re.split(r"\s+", line.strip())
    try:
        name = parts[0]
        filt = parts[5]
        time = float(parts[6])
        limit_flag = parts[8].startswith(">")
        mag = float(parts[8][1:]) if limit_flag else float(parts[8])
        mag_err = float(parts[9]) if not limit_flag else np.nan
        table1.append((name, filt, time, mag, mag_err, limit_flag))
    except Exception:
        continue

df1 = pd.DataFrame(table1, columns=["GRB", "Filter", "Time_since_burst_sec", "App_Mag", "Mag_Err", "Is_Limit"])
df1["GRB"] = df1["GRB"].str.extract(r"(GRB\d{6}[A-Z]?)")  # Normalize GRB names

# --- Load and clean Table 3 redshifts ---
with open("apj299693t3_ascii.txt", "r") as f:
    lines3 = f.readlines()

# Extract GRB and redshift columns from lines that begin with GRB
data_lines3 = [line for line in lines3 if line.startswith("GRB")]
table3 = []

for line in data_lines3:
    parts = re.split(r"\s+", line.strip())
    try:
        name = parts[0]
        z = float(parts[1])
        table3.append((name, z))
    except Exception:
        continue

df3 = pd.DataFrame(table3, columns=["GRB", "Redshift"])
df3["GRB"] = df3["GRB"].str.extract(r"(GRB\d{6}[A-Z]?)")  # Normalize GRB names

# --- Merge redshift into photometry ---
merged_df = pd.merge(df1, df3, on="GRB", how="left")

# --- Output ---
print(merged_df.head(20))  # Show 20 rows


FileNotFoundError: [Errno 2] No such file or directory: 'apj299693t1_mrt.txt'

In [31]:
import os
print(os.getcwd())


/Users/andradenebula/Documents/Research/Transient_Metrics/Multi_Transient_Metrics_Hub
